### Method 1

In [ ]:
main_dir = "../input/"
train_dir = "train/train"
path = os.path.join(main_dir, train_dir)

##########################################
### Loading 
##########################################

# Preprocess train data
X = []
y = []
convert = lambda category : int(category == 'dog')
def create_test_data(path):
    for p in os.listdir(path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)
    
create_test_data(path)
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

#import pickle
#pickle.dump( X, open( "train_x", "wb" ) )
#pickle.dump( y, open( "train_y", "wb" ) )   

#Normalize data
X = X/255.0

##########################################
### Model
##########################################

model = load_model('model.h5')

##########################################
### Evaluation
##########################################

# Preprocess test data similar to training data
train_dir = "test1/test1"
path = os.path.join(main_dir, train_dir)

X_test = []
id_line = []
def create_test1_data(path):
    for p in os.listdir(path):
        id_line.append(p.split(".")[0])
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
create_test1_data(path)
X_test = np.array(X_test).reshape(-1,80,80,1)
X_test = X_test/255

# Predictions
predictions = model.predict(X_test)
predicted_val = [int(round(p[0])) for p in predictions]
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})
submission_df[0:10]

### Method 2

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

def show_images(X_train, y_train, rows_num=6):    
    figure, axes = plt.subplots(nrows=rows_num, ncols=6, sharex=True, sharey=True, figsize=(20, rows_num * 2))
    
    for row_index in range(rows_num):
        ax_row = axes[row_index]
        images = X_train[(row_index*8):(row_index+1)*8]
        labels = y_train[(row_index*8):(row_index+1)*8]
        
        for ax, img, label in zip(ax_row, images, labels):
            ax.imshow(img)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            ax.set_title("Label - {}".format(label))

# Plotting model
f, ax = plt.subplots()
f.set_size_inches(10, 6)
    
ax.plot(history.history["loss"], c="r")
ax.plot(history.history["val_loss"], c="b")
ax.set_title("Error (Categorical Cross-Entropy)")
ax.set_ylabel("Value")
ax.set_xlabel("Epoch")
ax.legend(["Training", "Validation"], loc="best");

# Evaluation
test_score = model.evaluate(X_test, y_test, verbose=0)
test_score, test_accuracy = test_score
print("Test data results:")
print(" - error: {}".format(test_score))
print(" - acc: {}".format(test_accuracy))

### Method 3 - Getting filepath and loop pic in axis

In [ ]:
# Set directory
PATH = 'C:\\Users\\' + os.getlogin() + '\\Documents\\Programming\\DeepLearning\\09-dog-cat-classifier\\images'
os.chdir(PATH)

# Use Path
MAIN_PATH = Path(PATH)

# Dataframe
targets = list()
full_paths = list()

data = pd.DataFrame()
data['image_path'] = full_paths
data['target'] = targets

# Loop through file names
for i in FILE_NAMES:
    target = i.split('.')[0]
    full_path = os.path.join(MAIN_PATH / 'original_train', i)
    full_paths.append(full_path) # dog file location
    targets.append(target) # dog
    
def get_side(img, side_type, side_size=5):
    height, width, channel = img.shape
    if side_type=='horizontal':
        return np.ones((height, side_size, channel), dtype=np.float32)*255
    return np.ones((side_size, width, channel), dtype=np.float32)*255

def show_gallery(show='both'):
    n=100
    counter=0
    images=list()
    vertical_images=[]
    rng_state=np.random.get_state()
    np.random.shuffle(full_paths)
    np.random.set_state(rng_state)
    np.random.shuffle(targets)
    
    # Loop through images and target
    for path, target in zip(full_paths, targets): # combines both file & target together
        if target != show and show != 'both':
            continue
        counter=counter+1
        if counter%100==0:
            break
        
        # Load image
        img = load_img(path, target_size=(WIDTH, HEIGHT))
        
        # Convert img into array
        img = img_to_array(img)
        hside = get_side(img, side_type='horizontal')
        images.append(img)
        images.append(hside) 
        
        if counter % 10 == 0:
            himage = np.hstack((images))
            vside = get_side(himage, side_type='vertial')
            vertical_images.append(himage)
            vertical_images.append(vside)
            
            # Create images list
            images=list()
            
    gallery = np.vstack((vertical_images))
    plt.figure(figsize=(12, 12))
    plt.xticks([])
    plt.yticks([])
    title = {
        'both': 'Dogs and Cats',
        'cat': 'Cats',
        'dog': 'dogs'
    }
    plt.title('100 Samples of {} of the dataset'.format(title[show]))
    plt.imshow(gallery.astype(np.uint8))  

show_gallery(show='cat')
show_gallery(show='dog')
show_gallery()

### Method 4